In [1]:
def df_mean2(df):
  for dataset in df.columns.get_level_values(0).unique():
    df[dataset, 'promedio'] = df[dataset, 'mean_test_score'].apply(lambda x: np.mean(x))
  promedio_cols = [col for col in df.columns if col[1] == 'promedio']
  df[('overall', 'mean_datasets')] = df[promedio_cols].mean(axis=1)
  return df.sort_index(axis=1)

def get_df2(algorithm):
  with open(f'/content/drive/MyDrive/Colab Notebooks/regularizacion/JDEM/combinados_{algorithm}.pkl', 'rb') as f:
      df = pkl.load(f)
  return df_mean2(df)

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
import re
import pickle as pkl

def get_significance(p):
    if p < 0.001:
        return '^{***}'
    elif p < 0.01:
        return '^{**}'
    elif p < 0.05:
        return '^{*}'
    else:
        return ''


def extract_number(val):
    match = re.match(r"([0-9.]+)", str(val))
    return float(match.group(1)) if match else np.nan

resultados = {}
idx_max = {}
algoritmos = ['knn', 'rf', 'ridge', 'gb', 'adaboost', 'svc4']
datasets_names = ['ravdess', 'TESS', 'mesd', 'ShEMO', 'CaFE', 'EMO-DB', 'CREMA-D', 'EMOVO', 'EmoMatchSpanishDB', 'URDU']

for algorithm in algoritmos:
  resultados[algorithm] = get_df2(algorithm)
  idx_max[algorithm] = resultados[algorithm]['overall', 'mean_datasets'].idxmax()

summary_df = pd.DataFrame(index=algoritmos, columns=datasets_names)

for alg in algoritmos:
    for ds_name in datasets_names:
        values = resultados[alg].loc[idx_max[alg], (ds_name, 'mean_test_score')]
        mean_val = np.mean(values)*100
        if alg == "svc4":
            summary_df.loc[alg, ds_name] = f"{mean_val:.1f}"
        else:
            svc_vals = resultados['svc4'].loc[idx_max['svc4'], (ds_name, 'mean_test_score')]
            t_stat, p_val = ttest_rel(values, svc_vals)
            if alg == 'knn' and ds_name == 'mesd':
                print(f"{alg}, {ds_name}: {p_val}")
            stars = get_significance(p_val)
            summary_df.loc[alg, ds_name] = f"{mean_val:.1f} {stars}"

summary_df['overall'] = summary_df[datasets_names].map(extract_number).mean(axis=1).apply(lambda x: f"{x:.1f}")
for alg in algoritmos:
  values = summary_df[datasets_names].map(extract_number).loc[alg].to_list()
  if alg != 'svc4':
    svc_vals = summary_df[datasets_names].map(extract_number).loc['svc4'].to_list()
    t_stat, p_val = ttest_rel(values, svc_vals)
    if alg == 'knn':
      print(f"{alg}, overall: {p_val}")
    stars = get_significance(p_val)
    summary_df.loc[alg, 'overall'] += f" {stars}"
summary_df

knn, mesd: 0.8026144915120693
knn, overall: 0.19681248115197658


,ravdess,TESS,mesd,ShEMO,CaFE,EMO-DB,CREMA-D,EMOVO,EmoMatchSpanishDB,URDU,overall
knn,76.1,97.3 ^{*},77.4,54.1 ^{*},58.4,86.4,48.1 ^{***},84.2,56.2 ^{**},91.8,73.0
rf,66.2 ^{*},96.5 ^{*},70.2 ^{**},53.1 ^{***},56.6,81.1 ^{*},60.2 ^{*},82.5,59.6 ^{*},89.9,71.6 ^{**}
ridge,53.8 ^{**},89.7 ^{***},59.6 ^{**},56.2 ^{*},55.4,79.8 ^{***},54.3 ^{**},62.2 ^{**},53.9 ^{**},83.8 ^{*},64.9 ^{***}
gb,67.0 ^{*},96.7 ^{*},70.0 ^{*},55.3 ^{**},57.5,78.1 ^{*},59.4 ^{**},76.1 ^{**},61.0,85.6 ^{*},70.7 ^{***}
adaboost,51.0 ^{***},79.7 ^{**},48.9 ^{**},51.4 ^{**},49.3 ^{***},65.4 ^{*},57.7 ^{*},56.6 ^{**},49.4 ^{***},69.9 ^{*},57.9 ^{***}
svc4,73.0,97.9,77.0,58.8,58.3,87.0,61.6,83.2,64.2,91.3,75.2


In [ ]:
def bold_max_latex(col):
    max_val = col.map(lambda x: float(str(x).split()[0]) if isinstance(x, str) else x).max()
    return col.map(lambda x: f"\\textbf{{{x}}}" if str(x).startswith(f"{max_val:.1f}") else x)
df = summary_df.T.apply(bold_max_latex, axis=1)
df = df.applymap(lambda x: f"${x}$")
df


/tmp/ipython-input-32-1850857568.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: f"${x}$")


,knn,rf,ridge,gb,adaboost,svc4
ravdess,$\textbf{76.1 }$,$66.2 ^{*}$,$53.8 ^{**}$,$67.0 ^{*}$,$51.0 ^{***}$,$73.0$
TESS,$97.3 ^{*}$,$96.5 ^{*}$,$89.7 ^{***}$,$96.7 ^{*}$,$79.7 ^{**}$,$\textbf{97.9}$
mesd,$\textbf{77.4 }$,$70.2 ^{**}$,$59.6 ^{**}$,$70.0 ^{*}$,$48.9 ^{**}$,$77.0$
ShEMO,$54.1 ^{*}$,$53.1 ^{***}$,$56.2 ^{*}$,$55.3 ^{**}$,$51.4 ^{**}$,$\textbf{58.8}$
CaFE,$\textbf{58.4 }$,$56.6 $,$55.4 $,$57.5 $,$49.3 ^{***}$,$58.3$
EMO-DB,$86.4 $,$81.1 ^{*}$,$79.8 ^{***}$,$78.1 ^{*}$,$65.4 ^{*}$,$\textbf{87.0}$
CREMA-D,$48.1 ^{***}$,$60.2 ^{*}$,$54.3 ^{**}$,$59.4 ^{**}$,$57.7 ^{*}$,$\textbf{61.6}$
EMOVO,$\textbf{84.2 }$,$82.5 $,$62.2 ^{**}$,$76.1 ^{**}$,$56.6 ^{**}$,$83.2$
EmoMatchSpanishDB,$56.2 ^{**}$,$59.6 ^{*}$,$53.9 ^{**}$,$61.0 $,$49.4 ^{***}$,$\textbf{64.2}$
URDU,$\textbf{91.8 }$,$89.9 $,$83.8 ^{*}$,$85.6 ^{*}$,$69.9 ^{*}$,$91.3$


In [ ]:
citas = {'ravdess': '\\citep{ravdess}', 'TESS': '\\citep{tess}', 'mesd': '\\citep{duville2021mexican}', 'ShEMO': '\\citep{shemo}', 'CaFE': '\\citep{cafe}', 'EMO-DB': '\\citep{emodb}',
 'CREMA-D': '\\citep{crema}','EMOVO': '\\citep{emovo}', 'EmoMatchSpanishDB': '\\citep{garcia2024emomatchspanishdb}', 'URDU': '\\citep{urdu}', 'overall': '\\citep{overall}'}

df.index = df.index.map(citas)
df

,knn,rf,ridge,gb,adaboost,svc4
\citep{ravdess},$\textbf{76.1 }$,$66.2 ^{*}$,$53.8 ^{**}$,$67.0 ^{*}$,$51.0 ^{***}$,$73.0$
\citep{tess},$97.3 ^{*}$,$96.5 ^{*}$,$89.7 ^{***}$,$96.7 ^{*}$,$79.7 ^{**}$,$\textbf{97.9}$
\citep{duville2021mexican},$\textbf{77.4 }$,$70.2 ^{**}$,$59.6 ^{**}$,$70.0 ^{*}$,$48.9 ^{**}$,$77.0$
\citep{shemo},$54.1 ^{*}$,$53.1 ^{***}$,$56.2 ^{*}$,$55.3 ^{**}$,$51.4 ^{**}$,$\textbf{58.8}$
\citep{cafe},$\textbf{58.4 }$,$56.6 $,$55.4 $,$57.5 $,$49.3 ^{***}$,$58.3$
\citep{emodb},$86.4 $,$81.1 ^{*}$,$79.8 ^{***}$,$78.1 ^{*}$,$65.4 ^{*}$,$\textbf{87.0}$
\citep{crema},$48.1 ^{***}$,$60.2 ^{*}$,$54.3 ^{**}$,$59.4 ^{**}$,$57.7 ^{*}$,$\textbf{61.6}$
\citep{emovo},$\textbf{84.2 }$,$82.5 $,$62.2 ^{**}$,$76.1 ^{**}$,$56.6 ^{**}$,$83.2$
\citep{garcia2024emomatchspanishdb},$56.2 ^{**}$,$59.6 ^{*}$,$53.9 ^{**}$,$61.0 $,$49.4 ^{***}$,$\textbf{64.2}$
\citep{urdu},$\textbf{91.8 }$,$89.9 $,$83.8 ^{*}$,$85.6 ^{*}$,$69.9 ^{*}$,$91.3$


In [ ]:
latex_code = df.to_latex(escape=False)
latex_code

'\\begin{tabular}{lllllll}\n\\toprule\n & knn & rf & ridge & gb & adaboost & svc4 \\\\\n\\midrule\n\\citep{ravdess} & $\\textbf{76.1 }$ & $66.2 ^{*}$ & $53.8 ^{**}$ & $67.0 ^{*}$ & $51.0 ^{***}$ & $73.0$ \\\\\n\\citep{tess} & $97.3 ^{*}$ & $96.5 ^{*}$ & $89.7 ^{***}$ & $96.7 ^{*}$ & $79.7 ^{**}$ & $\\textbf{97.9}$ \\\\\n\\citep{duville2021mexican} & $\\textbf{77.4 }$ & $70.2 ^{**}$ & $59.6 ^{**}$ & $70.0 ^{*}$ & $48.9 ^{**}$ & $77.0$ \\\\\n\\citep{shemo} & $54.1 ^{*}$ & $53.1 ^{***}$ & $56.2 ^{*}$ & $55.3 ^{**}$ & $51.4 ^{**}$ & $\\textbf{58.8}$ \\\\\n\\citep{cafe} & $\\textbf{58.4 }$ & $56.6 $ & $55.4 $ & $57.5 $ & $49.3 ^{***}$ & $58.3$ \\\\\n\\citep{emodb} & $86.4 $ & $81.1 ^{*}$ & $79.8 ^{***}$ & $78.1 ^{*}$ & $65.4 ^{*}$ & $\\textbf{87.0}$ \\\\\n\\citep{crema} & $48.1 ^{***}$ & $60.2 ^{*}$ & $54.3 ^{**}$ & $59.4 ^{**}$ & $57.7 ^{*}$ & $\\textbf{61.6}$ \\\\\n\\citep{emovo} & $\\textbf{84.2 }$ & $82.5 $ & $62.2 ^{**}$ & $76.1 ^{**}$ & $56.6 ^{**}$ & $83.2$ \\\\\n\\citep{garcia2024em